# 1. Paralelní výpočty v Cythonu (OpenMP)

V této části navazujeme na minulou lekci a ukážeme si dvě praktické cesty, jak v Cythonu využít více vláken na CPU:

- explicitní paralelní blok `with nogil, parallel():`
- paralelní smyčku přes `prange()`

## 1.1 Paralelní blok `with nogil, parallel():`

Jde o Cython obálku nad OpenMP direktivou pro paralelní sekci. Vlákna se vytvoří při vstupu do bloku a po jeho skončení se ukončí.

Uvnitř bloku:
- každé vlákno má vlastní kontext lokálních proměnných,
- se sdílenými daty je potřeba pracovat opatrně,
- identifikaci vlákna získáme přes `omp_get_thread_num()`.

## 1.2 Paralelní smyčka přes `prange()`

`prange()` je paralelní alternativa k `range()`. Rozdělení iterací mezi vlákna řídí OpenMP.

Důležité argumenty:
- `num_threads`: počet vláken,
- `schedule`: strategie rozdělení práce,
- `chunksize`: velikost bloku iterací,
- `nogil=True`: běh bez GIL.

Operátor `+=` uvnitř `prange` znamená redukci do sdíleného výsledku. To je užitečné, ale je potřeba s tím počítat při návrhu algoritmu.

In [ ]:
# pro jednoduchost použijeme Cython magic přímo v notebooku
%load_ext cython

## 1.3 Příklad: norma vektoru

### 1.3.1 Sekvenční verze

In [ ]:
%%cython --compile-args=-O3
import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm_serial(np.ndarray[np.float64_t, ndim=1] a):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in range(n):
        result += a[i] * a[i]
    return sqrt(result)

In [ ]:
import numpy as np
x = np.random.rand(4_000_000)
y1 = my_norm_serial(x)
y2 = np.linalg.norm(x)
print(y1, y2)

In [ ]:
%timeit _ = np.linalg.norm(x)

In [ ]:
%timeit _ = my_norm_serial(x)

### 1.3.2 Paralelizace přes `with nogil, parallel():`

In [ ]:
%%cython --compile-args=-fopenmp --compile-args=-O3 --link-args=-fopenmp

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython

from cython.parallel import parallel
from openmp cimport omp_get_thread_num

ctypedef np.float64_t DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm_parallel_block(np.ndarray[np.float64_t, ndim=1] a, int num_threads):
    cdef int i
    cdef int n = a.shape[0]
    cdef int chunk_size = n // num_threads
    cdef int thread_num, start_idx, end_idx
    cdef double local_sum
    cdef double result = 0.0
    cdef np.ndarray[np.float64_t, ndim=1] partial_sums = np.zeros((num_threads), dtype=np.float64)

    with nogil, parallel(num_threads=num_threads):
        thread_num = omp_get_thread_num()
        start_idx = thread_num * chunk_size
        if thread_num == num_threads - 1:
            end_idx = n
        else:
            end_idx = (thread_num + 1) * chunk_size

        local_sum = 0.0
        for i in range(start_idx, end_idx):
            local_sum = local_sum + a[i] * a[i]
        partial_sums[thread_num] = local_sum

    for i in range(num_threads):
        result += partial_sums[i]

    return sqrt(result)

In [ ]:
x = np.random.rand(4_000_000)
y1 = my_norm_parallel_block(x, 4)
y2 = np.linalg.norm(x)
print(y1, y2)

In [ ]:
%timeit _ = my_norm_parallel_block(x, 8)

In [ ]:
%timeit _ = np.linalg.norm(x)

### 1.3.3 Paralelizace přes `prange()`

In [ ]:
%%cython --compile-args=-fopenmp --compile-args=-O3 --link-args=-fopenmp

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
import cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double my_norm_parallel_prange(np.ndarray[np.float64_t, ndim=1] a, int num_threads):
    cdef int i
    cdef int n = a.shape[0]
    cdef double result = 0.0
    for i in prange(n, nogil=True, num_threads=num_threads):
        result += a[i] * a[i]
    return sqrt(result)

In [ ]:
x = np.random.rand(4_000_000)
y1 = my_norm_parallel_prange(x, 4)
y2 = np.linalg.norm(x)
print(y1, y2)

In [ ]:
%timeit _ = my_norm_parallel_prange(x, 8)

In [ ]:
%timeit _ = np.linalg.norm(x)

## 1.4 Srovnání rychlosti

In [ ]:
import os
import time
import matplotlib.pyplot as plt


def measure_multi(n, func, data):
    tmp_time = []
    for _ in range(n):
        start = time.time()
        _ = func(data)
        tmp_time.append(time.time() - start)
    return min(tmp_time)

n_loops = 5
x = np.random.rand(2_000_000)

max_threads = min(32, os.cpu_count() or 1)
pocet_vlaken = []
threads = 1
while threads <= max_threads:
    pocet_vlaken.append(threads)
    threads *= 2

time_numpy = measure_multi(n_loops, lambda data: np.linalg.norm(data), x)

time_parallel_block = []
time_parallel_prange = []

for n_threads in pocet_vlaken:
    time_parallel_block.append(
        measure_multi(n_loops, lambda data: my_norm_parallel_block(data, n_threads), x)
    )
    time_parallel_prange.append(
        measure_multi(n_loops, lambda data: my_norm_parallel_prange(data, n_threads), x)
    )

    print(
        f"vlaken: {n_threads}, with parallel(): {time_parallel_block[-1]:.6f}s, "
        f"prange: {time_parallel_prange[-1]:.6f}s"
    )

plt.loglog(pocet_vlaken, [time_numpy for _ in pocet_vlaken], label="numpy")
plt.loglog(pocet_vlaken, time_parallel_block, label="with parallel()")
plt.loglog(pocet_vlaken, time_parallel_prange, label="prange()")
plt.xlabel("Počet vláken")
plt.ylabel("Čas [s]")
plt.grid()
plt.legend()